In [ ]:
'''
compute semantic similarity between source library and target library
'''

In [ ]:
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from itertools import product
import pandas as pd
import math

In [ ]:

bnc_ic = wordnet_ic.ic('ic-bnc-resnik-add1.dat')

In [ ]:
df = pd.read_excel(r'../resources/labelSupport.xlsx')
fromList = list(df['fromLabels'])
toList = list(df['toLabels'])
fromLabels=[]
toLabels=[]
for item in fromList:
    fromLabels.append(item.split('/'))
for item in toList:
    toLabels.append(item.split('/'))



In [ ]:
fromSet = sorted(set(fromList),key=fromList.index)
toSet = sorted(set(toList),key=toList.index)
print(len(fromSet))
print(len(toSet))


In [ ]:
labelList=[]
for item in fromSet:
    labelList.append(item.split('/'))
for item in toSet:
    labelList.append(item.split('/'))
print(len(labelList))


In [ ]:

def precompute_synsets(labelSentence,syn_dict):
    temp=[]
    for word in labelSentence:
        syns = wn.synsets(word)
        if len(syns)==0:
            continue
        for syn in syns:
            temp.append(syn)
    syn_dict[labelSentence]=temp
    

In [ ]:
syn_dict = {}
for item in labelList:
    precompute_synsets(tuple(item),syn_dict)


In [ ]:
def compute_similarity(labelSentence1,labelSentence2,result):
    scores = []
    syns1 = syn_dict[tuple(labelSentence1)]   
    syns2 = syn_dict[tuple(labelSentence2)]  
    for sense1, sense2 in product(syns1, syns2):
        if sense1.pos() not in ['n','v'] or sense2.pos() not in ['n','v']:
            continue
        if sense1.pos()!=sense2.pos():
            continue
        score = wn.jcn_similarity(sense1, sense2,bnc_ic)
        scores.append(score)
    if len(scores)!=0:
        similarity = max(scores)
        result.append(similarity)       
    else:
        result.append(1)
    

In [ ]:
result = []
length = len(fromLabels)
for i in range(length):
    compute_similarity(fromLabels[i],toLabels[i],result)
print(len(result))  

In [ ]:
print(result[:10])

In [ ]:
import pandas as pd
data = result
df = pd.DataFrame(data,columns=['similarity'])
df.to_excel(r'../resources/wordNet.xlsx', index=False)


In [ ]:
'''
generate confidence2
'''

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel(r'../resources/wordnet.xlsx')
confidenceList = list(df['confidence'])
similarityList = list(df['similarity'])
labelSupportList = list(df['labelSupport'])
formattedList = []


In [ ]:
length = len(similarityList)
print(length)

In [ ]:
for i in range(length):
    if similarityList[i]==1e+300:
        formattedList.append(labelSupportList[i])
    else:
        formattedList.append(similarityList[i])

In [ ]:
print(len(formattedList))
result = []
for i in range(length):
    result.append(confidenceList[i]*formattedList[i])


In [ ]:
data = result
df = pd.DataFrame(data,columns=['confidence2'])
df.to_excel(r'../resources/wordNet.xlsx', index=False)